<img src="SalesKen.png" alt="upGrad" align="Left" style="width: 200px;"/>

## SalesKen Data Science Test
<i><b>Author: Anish Mahapatra</i></b>

<i>Date: 01-Feb-2019</i>

Problem Statement - 1: Matching the misspelt cities.
There are two data files (.csv)

<b>Correct_cities.csv</b> : This file consists of a list of cities and they are spelt correctly. It has three columns "name" which is a city name; "country" where the city belongs to and "id" which is a unique id for each city.

<b>Misspelt_cities.csv</b> : This file consists of city names which are mispelt. Each city name has been misspelt by randomly replacing 20% of the characters by some random characters. It has two columns "misspelt_name" and "country".

Question : Write an algorithm or a group of algorithms to match the misspelt city name with the correct city name and return the list of unique ids for each misspelt city name.

For example : Correct Data -> (Bangalore, India, 101) and say for "Bangalore" the misspelt name is (Banhakorg, India). Then the matching algorithm should return 101 for this example.

<b>Approach 1:</b>

Steps to be followed to find and perform partial match:
- Read the correct_cities.csv and misspelt_cities.csv file as a pandas dataframe
    - Sense check of the data
    - Check the shape of the data
    - check for missing values (NaN)
- By country, iterate through the cities and compute the similarity score (Levensthein using fuzzywuzzy)
    - Substeps for similarity: convert to lower case, find the partial match score
- Set the threshold for the similariity score
- If the similarity score is greater than threshold, assign the id from the correct_cities onto the misspelt_cities file

<b>Next Steps:</b>
1. Look at the misspelt cities that still do not have an id
2. Use a different algorithm to perform partial text match on the files


In [1]:
# Packages to be imported
import matplotlib.pyplot as plt
from fuzzywuzzy import process
from fuzzywuzzy import fuzz 
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
# Reading the csv files
correctCities = pd.read_csv("Correct_cities.csv")
misspeltCities = pd.read_csv("Misspelt_cities.csv")

## Correct cities

In [3]:
# Sense check of the data
correctCities.head()

,name,country,id
0,les Escaldes,Andorra,3040051
1,Andorra la Vella,Andorra,3041563
2,Umm al Qaywayn,United Arab Emirates,290594
3,Ras al-Khaimah,United Arab Emirates,291074
4,Khawr Fakkān,United Arab Emirates,291696


In [4]:
# Shape of the data
correctCities.shape

(23018, 3)

In [5]:
# Checking for missing values
correctCities.isnull().sum(axis = 0)

name       0
country    0
id         0
dtype: int64

## Misspelt Cities

In [6]:
# Sense check of the data
misspeltCities.head()

,misspelt_name,country
0,Hfjdúszoposzló,Hungary
1,Otrajnyy,Russia
2,ian Isidre,Peru
3,Bordj Zemoufa,Algeria
4,ChulamViwta,United States


In [7]:
misspeltCities.shape

(23018, 2)

In [8]:
# Checking for missing values
misspeltCities.isnull().sum(axis = 0)

misspelt_name    0
country          0
dtype: int64

Awesome! 

Now we know that the shape of the data for both the data frames is the same and we should be able to match the correct files with the corrupted ones.

##### Let's convert the misspelt names and correct names to lowercase to ensure that the scores have inegrity

In [9]:
# Converting the column to lower case'
correctCities['name'].str.lower()

0            les escaldes
1        andorra la vella
2          umm al qaywayn
3          ras al-khaimah
4            khawr fakkān
               ...       
23013            bulawayo
23014             bindura
23015          beitbridge
23016             epworth
23017         chitungwiza
Name: name, Length: 23018, dtype: object

In [10]:
# Converting the column to lower case'
misspeltCities['misspelt_name'].str.lower()

0        hfjdúszoposzló
1              otrajnyy
2            ian isidre
3         bordj zemoufa
4           chulamviwta
              ...      
23013        zdigulegsk
23014            ygring
23015         moudougou
23016            bhīkxi
23017    belvfbere park
Name: misspelt_name, Length: 23018, dtype: object

Let's now feed this back to the original dataframe

In [11]:
# Feeding lower case back to original dataframe
correctCities['name'] = correctCities['name'].str.lower()
misspeltCities['misspelt_name'] = misspeltCities['misspelt_name'].str.lower()

### Fuzzywuzzy

In [12]:
# Sense check to generate the score between two strings
fuzz.ratio("SalesKen","Slaeeenek")

59

In [13]:
# Sense check to generate the partial score between two strings
fuzz.partial_ratio("SalesKen","Slaeeenek")

62

We shall be using the **fuzz.ratio** function as the order of the cities will remain the same 

In [14]:
# Function to check get the scores
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

Taking the columns to be matched and converting them to a list

Styling:

In [15]:
%%HTML
<style>.dataframe th, td:first-child{background:#3f577c;font-family:monospace;color:white;border:3px solid white;
text-align:left !important;}#codex{float:right;}</style>

In [16]:
correctCities.head(3)

,name,country,id
0,les escaldes,Andorra,3040051
1,andorra la vella,Andorra,3041563
2,umm al qaywayn,United Arab Emirates,290594


In [17]:
misspeltCities.head(3)

,misspelt_name,country
0,hfjdúszoposzló,Hungary
1,otrajnyy,Russia
2,ian isidre,Peru


In [18]:
# Reading a sample csv file
df = pd.read_csv('test.csv')

In [19]:
# Sense check of the data
df.head(2)

,correctname,wrongcityname


In [20]:
# Making a new dataframe, where we shall add the similarity score 
df['correctname'] = correctCities['name']
df['wrongcityname'] = misspeltCities['misspelt_name']

In [21]:
# # Using the function to match 
# name_match,ratio_match=checker(misspeltCitiesList,correctCitiesList)

In [22]:
# df1 = pd.DataFrame()
# df1['old_names']=pd.Series(str2Match)
# df1['correct_names']=pd.Series(name_match)
# df1['correct_ratio']=pd.Series(ratio_match)
# df1.to_excel('matched_names.xlsx', engine='xlsxwriter')

Here, we have tried two approaches:
1. Using the Levensthein formula 
2. Using the FuzzyWuzzy package

The output of the code is to give us a similarity score. We should iterate it by country, else it becomes inefficient as it ha a score of O(n^2) 

It is taking more time than we have, as a competency showcase, let us try to do it for the country Australia

In [23]:
australia = pd.read_csv('Australia.csv')

In [24]:
australia.head()

,correctname,country,correctid,wrongname
0,Whyalla,Australia,2058430,Esrlwood
1,Rockingham,Australia,2062338,Gleoroy
2,Prospect,Australia,2062944,Sundury
3,Port Hedland,Australia,2063042,Armakale
4,Perth,Australia,2063523,Caloundfa


In [25]:
# Function to get the similarity score of the correct city name and the wrong city name
correct_name = []
similarity = []

for i in australia.correctname:
    ratio = process.extract( i, australia.wrongname, limit=1)
    correct_name.append(ratio[0][0])
    similarity.append(ratio[0][1])

In [26]:
# Adding the similarity score
australia['similarity'] = pd.Series(similarity)

In [27]:
final_result_australia = australia[['correctname', 'country', 'correctid', 'wrongname', 'similarity']]
final_result_australia.head(3)

,correctname,country,correctid,wrongname,similarity
0,Whyalla,Australia,2058430,Esrlwood,86
1,Rockingham,Australia,2062338,Gleoroy,80
2,Prospect,Australia,2062944,Sundury,88


In [28]:
final_result_australia.head(8).style.background_gradient(subset='similarity',
                                               cmap='summer_r')

,correctname,country,correctid,wrongname,similarity
0,Whyalla,Australia,2058430,Esrlwood,86
1,Rockingham,Australia,2062338,Gleoroy,80
2,Prospect,Australia,2062944,Sundury,88
3,Port Hedland,Australia,2063042,Armakale,83
4,Perth,Australia,2063523,Caloundfa,80
5,Murray Bridge,Australia,2065176,Craigibbzrn,85
6,Mount Isa,Australia,2065594,Melbournk,89
7,Morphett Vale,Australia,2065740,South Grscton,92


In [29]:
australia.head()

,correctname,country,correctid,wrongname,similarity
0,Whyalla,Australia,2058430,Esrlwood,86
1,Rockingham,Australia,2062338,Gleoroy,80
2,Prospect,Australia,2062944,Sundury,88
3,Port Hedland,Australia,2063042,Armakale,83
4,Perth,Australia,2063523,Caloundfa,80


In [30]:
hashdict = dict(zip(australia['correctid'], australia['correctname']))

def checkpair (a,b,l):
    for x in l:
        if (a,b) == (x[2],x[0]):
            l.remove(x)

matches = []
for k,v in hashdict.items():

    #see docs for extract -- 4 because you are comparing a name to itself
    top3 = process.extract(v, hashdict, limit=4)

    #remove the hashID compared to itself
    for h in top3:
        if k == h[2]:
            top3.remove(h)

    #append tuples to the list "matches" if it meets a score criteria      
    [matches.append((k, v, x[2], x[0], x[1])) for x in top3 if x[1] > 60] #change score?

    #remove reciprocal pairs
    [checkpair(m[0], m[2], matches) for m in matches]

df = pd.DataFrame(matches, columns=['original_id', 'correct_city_name', 'mapped_id', 'misspelt_city', 'score'])
#writer = pd.ExcelWriter('C:\\Users\\HP\\Desktop\\SalesKen Test\\SaleskenProblemSolving-master\\03AustraliaAnswer.xlsx')
# df.to_excel(writer,'Sheet1')
# writer.save()

In [31]:
df.head()

,original_id,correct_city_name,mapped_id,misspelt_city,score
0,2062338,Rockingham,2151437,Rockhampton,67
1,2063042,Port Hedland,2148398,Port Stephens,64
2,2065176,Murray Bridge,6943558,Bracken Ridge,62
3,2065594,Mount Isa,2156663,Mount Eliza,80
4,2065594,Mount Isa,2156578,Mount Martha,67


Great, we have now got a similarity score for Australia!

Now, we need to map the correctid, which is the id of the correct cities to the wrong name

Let's now try a different approach to get the unique ids
def checkpair (a,b,l):
    for x in l:
        if (a,b) == (x[2],x[0]):
            l.remove(x)

In [32]:
df = australia.copy(deep=True)

In [33]:
df.head()

,correctname,country,correctid,wrongname,similarity
0,Whyalla,Australia,2058430,Esrlwood,86
1,Rockingham,Australia,2062338,Gleoroy,80
2,Prospect,Australia,2062944,Sundury,88
3,Port Hedland,Australia,2063042,Armakale,83
4,Perth,Australia,2063523,Caloundfa,80


In [34]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

df1=df['correctname']
df2=df['wrongname']

df1.columns=['correctname']
df2.columns=['wrongname']

def match(Col1,Col2):
    overall=[]
    for n in Col1:
        result=[(fuzz.partial_ratio(n, n2),n2) 
                for n2 in Col2 if fuzz.partial_ratio(n, n2)>50
               ]
        if len(result):
            result.sort()    
            print('result {}'.format(result))
            print("Best M={}".format(result[-1][1]))
            overall.append(result[-1][1])
        else:
            overall.append(" ")
    return overall

print(match(df1,df2))

result [(86, 'Whyrlla')]
Best M=Whyrlla
result [(60, 'kmina'), (80, 'Rocpbngham')]
Best M=Rocpbngham
result [(88, 'Pronpect')]
Best M=Pronpect
result [(53, 'Adelakde'), (56, 'St qlbans'), (60, 'Pertx'), (83, 'Pork Hedlanw')]
Best M=Pork Hedlanw
result [(60, 'Caeberwhll'), (60, 'Cheltdhham'), (60, 'Ferntree dwlly'), (60, 'NoblevParh'), (60, 'Northcotp'), (60, 'Nxrth fyde'), (60, 'PortwStlphens'), (60, 'Willethon'), (60, 'palmermton'), (80, 'Pertx')]
Best M=Pertx
result [(56, 'Wotdridge'), (85, 'burrav Bridge')]
Best M=burrav Bridge
result [(56, 'South Grscton'), (67, 'Moe'), (67, 'Mount Elwda'), (67, 'iount Galbier'), (78, 'Mount Martrj'), (89, 'MountmIsa')]
Best M=MountmIsa
result [(55, 'Forest aoke'), (67, 'Moe'), (92, 'MorphetutVale')]
Best M=MorphetutVale
result [(57, 'Sundury'), (57, 'qildura'), (88, 'Mabdurah')]
Best M=Mabdurah
result [(60, 'kmina'), (67, 'Kew'), (100, 'Kwinana')]
Best M=Kwinana
result [(60, 'Lalon'), (80, 'Ktlgofrlie')]
Best M=Ktlgofrlie
result [(88, 'Gvsnells')]

# Problem 2

Part - 1: Given a list of sentences (list_of_setences.txt) write an algorithm which computes the semantic similarity and return the similar sentences together.

Semantic similarity is a metric defined over a set of documents or terms, where the idea of distance between them is based on the likeness of their meaning or semantic content.

For example : "Football is played in Brazil" and "Cricket is played in India". Both these sentences are about sports so they will have a semantic similarity.

Part - 2: Extend the above algorithm in form of a REST API. The input parameter is a list of sentences (refer to the file list_of_setences.txt) and the response is a list of list with the similar sentences.

For example : Say there are 4 sentences as an input list - ["Football is played in Brazil" , "Cricket is played in India", "Traveling is good for health", "People love traveling in winter"]

Output : [["Football is played in Brazil" , "Cricket is played in India"], ["Traveling is good for health", "People love traveling in winter"]]

In [35]:
from nltk.corpus import wordnet as wn
from subprocess import check_output
from nltk import word_tokenize
import pandas as pd
import numpy as np
import math
import nltk
import re
from nltk import word_tokenize

In [36]:
# listOfSentences = pd.read_csv('list_of_sentences.txt')
documents = [
    "good morning",
    "how are you doing ?",
    "the weather is awesome today",
    "samsung",
    "good afternoon",
    "baseball is played in the USA",
    "there is a thunderstorm",
    "are you doing good ?",
    "The polar regions are melting",
    "apple",
    "nokia",
    "cricket is a fun game",
    "the climate change is a problem",
]

In [37]:
documents

['good morning',
 'how are you doing ?',
 'the weather is awesome today',
 'samsung',
 'good afternoon',
 'baseball is played in the USA',
 'there is a thunderstorm',
 'are you doing good ?',
 'The polar regions are melting',
 'apple',
 'nokia',
 'cricket is a fun game',
 'the climate change is a problem']

## Approach 1:

Using gensim

In [38]:
from collections import defaultdict
from gensim import corpora

In [39]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [40]:
corpus

[[(0, 1)],
 [(1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1)],
 [],
 [(0, 1)],
 [(5, 1)],
 [(5, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(2, 1)],
 [],
 [],
 [(5, 1)],
 [(5, 1)]]

In [41]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=10)

In [42]:
doc = "cricket is a fun game"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(1, -0.9999999999999994)]


In [43]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

In [44]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.0), (1, 0.0), (2, 1.0), (3, 0.0), (4, 0.0), (5, 1.0), (6, 1.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 1.0), (12, 1.0)]


In [45]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in enumerate(sims):
    print(s, documents[i])

(2, 1.0) good morning
(5, 1.0) how are you doing ?
(6, 1.0) the weather is awesome today
(11, 1.0) samsung
(12, 1.0) good afternoon
(0, 0.0) baseball is played in the USA
(1, 0.0) there is a thunderstorm
(3, 0.0) are you doing good ?
(4, 0.0) The polar regions are melting
(7, 0.0) apple
(8, 0.0) nokia
(9, 0.0) cricket is a fun game
(10, 0.0) the climate change is a problem


## Approcah 2: --ignored

In [ ]:
import fastsemsim
str1='Birthday party ruined as cake explodes'
str2='Grandma mistakenly bakes cake using gunpowder'

In [ ]:
Semsim(str1,str2)

In [ ]:
def tokenize(q1, q2):
    """
        q1 and q2 are sentences/questions. Function returns a list of tokens for both.
    """
    return word_tokenize(q1), word_tokenize(q2)


def posTag(q1, q2):
    """
        q1 and q2 are lists. Function returns a list of POS tagged tokens for both.
    """
    return nltk.pos_tag(q1), nltk.pos_tag(q2)

def stemmer(tag_q1, tag_q2):
    """
        tag_q = tagged lists. Function returns a stemmed list.
    """

    stem_q1 = []
    stem_q2 = []

    for token in tag_q1:
        stem_q1.append(stem(token))

    for token in tag_q2:
        stem_q2.append(stem(token))

    return stem_q1, stem_q2

### Micheal Lesk Algorithm
We are going to use the Micheal Lesk Algorithm for word sense disambugation (WSD). It uses WordNet.

- It excludes stop words
- The overlap is calculated using the gloss of all the context words such that the most matching sense of the word will be accepted as the best sense of the word in the context.

In [ ]:

    def __init__(self, sentence):
        self.sentence = sentence
        self.meanings = {}
        for word in sentence:
            self.meanings[word] = ''

    def getSenses(self, word):
        # print word
        return wn.synsets(word.lower())

    def getGloss(self, senses):

        gloss = {}

        for sense in senses:
            gloss[sense.name()] = []

        for sense in senses:
            gloss[sense.name()] += word_tokenize(sense.definition())

        return gloss

    def getAll(self, word):
        senses = self.getSenses(word)

        if senses == []:
            return {word.lower(): senses}

        return self.getGloss(senses)

    def Score(self, set1, set2):
        # Base
        overlap = 0

        # Step
        for word in set1:
            if word in set2:
                overlap += 1

        return overlap

    def overlapScore(self, word1, word2):

        gloss_set1 = self.getAll(word1)
        if self.meanings[word2] == '':
            gloss_set2 = self.getAll(word2)
        else:
            # print 'here'
            gloss_set2 = self.getGloss([wn.synset(self.meanings[word2])])

        # print gloss_set2

        score = {}
        for i in gloss_set1.keys():
            score[i] = 0
            for j in gloss_set2.keys():
                score[i] += self.Score(gloss_set1[i], gloss_set2[j])

        bestSense = None
        max_score = 0
        for i in gloss_set1.keys():
            if score[i] > max_score:
                max_score = score[i]
                bestSense = i

        return bestSense, max_score

    def lesk(self, word, sentence):
        maxOverlap = 0
        context = sentence
        word_sense = []
        meaning = {}

        senses = self.getSenses(word)

        for sense in senses:
            meaning[sense.name()] = 0

        for word_context in context:
            if not word == word_context:
                score = self.overlapScore(word, word_context)
                if score[0] == None:
                    continue
                meaning[score[0]] += score[1]

        if senses == []:
            return word, None, None

        self.meanings[word] = max(meaning.keys(), key=lambda x: meaning[x])

        return word, self.meanings[word], wn.synset(self.meanings[word]).definition()

Two different similarity measures have been used:
1. Path similarity which calculates the distance in the words in the hyponym taxonymy graph
2. Wup similarity is the Wu & Palmer's similarity

In [ ]:
import numpy as np
from scipy import spatial
from nltk.corpus import wordnet as wn
from nltk.metrics import edit_distance

def path(set1, set2):
    return wn.path_similarity(set1, set2)


def wup(set1, set2):
    return wn.wup_similarity(set1, set2)


def edit(word1, word2):
    if float(edit_distance(word1, word2)) == 0.0:
        return 0.0
    return 1.0 / float(edit_distance(word1, word2))

In [ ]:
def computePath(q1, q2):

    R = np.zeros((len(q1), len(q2)))

    for i in range(len(q1)):
        for j in range(len(q2)):
            if q1[i][1] == None or q2[j][1] == None:
                sim = edit(q1[i][0], q2[j][0])
            else:
                sim = path(wn.synset(q1[i][1]), wn.synset(q2[j][1]))

            if sim == None:
                sim = edit(q1[i][0], q2[j][0])

            R[i, j] = sim

    # print R

    return R

In [ ]:
def computeWup(q1, q2):

    R = np.zeros((len(q1), len(q2)))

    for i in range(len(q1)):
        for j in range(len(q2)):
            if q1[i][1] == None or q2[j][1] == None:
                sim = edit(q1[i][0], q2[j][0])
            else:
                sim = wup(wn.synset(q1[i][1]), wn.synset(q2[j][1]))

            if sim == None:
                sim = edit(q1[i][0], q2[j][0])

            R[i, j] = sim

    # print R

    return R

This is the fifth part of the algorithm mentioned below. Combines the similarity measures calculated for the two sentences and produces a single similarity score.

In [ ]:
def overallSim(q1, q2, R):

    sum_X = 0.0
    sum_Y = 0.0

    for i in range(len(q1)):
        max_i = 0.0
        for j in range(len(q2)):
            if R[i, j] > max_i:
                max_i = R[i, j]
        sum_X += max_i

    for i in range(len(q1)):
        max_j = 0.0
        for j in range(len(q2)):
            if R[i, j] > max_j:
                max_j = R[i, j]
        sum_Y += max_j
        
    if (float(len(q1)) + float(len(q2))) == 0.0:
        return 0.0
        
    overall = (sum_X + sum_Y) / (2 * (float(len(q1)) + float(len(q2))))

    return overall